In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
import json, re, nltk
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

from nltk.stem.wordnet import WordNetLemmatizer
Lem = WordNetLemmatizer()

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("talk")

In [3]:
scopus = pd.read_csv("scopus.csv")
keyword_org_name = "keywords.txt"
keyword_abb_name = "keywords_abb.json"
keyword_dash_name = "keywords_dash.json"
keyword_dict_name = "keywords_dict.json"
keyword_single_name = "keywords_single.txt"
keyword_plural_name = "keywords_plural.json"
unicode_name = "unicode_json"

In [4]:
# unicode dictionary
with open('unicode_dict.json') as j:
    unicode_dict = json.load(j)

In [5]:
keywords = []
line = ''

with open("BIPV_ML_all.txt") as f:
    while line != "EF":
        line = f.readline()
        if line[:3] == "DE ":
            keywords_new = line[3:].rstrip('\n').split(";")
            for keyword_new in keywords_new:
                keywords += [keyword_new.rstrip(' ').lstrip(' ')]

keywords = sorted(keywords)
with open(keyword_org_name, "w", encoding="utf-8-sig") as f:
    for keyword in keywords:
        f.write(f"{keyword}\n")

In [6]:
# retreive keywords
keywords_raw_ = np.genfromtxt("keywords.txt", dtype="str", delimiter="\n", encoding='utf-8-sig') #utf-8-sig for \ufeff removal

# get unique ones
keywords_raw = np.unique(keywords_raw_)

# abbriviations detection (key candidates)
keywords_abb = {}
keywords_abb_key_cand = []
for keyword_raw_ in keywords_raw_:
    kwr = keyword_raw_.split()
    for kw in kwr:
        kw = kw.lstrip("(").rstrip(")")
        if (kw == kw.upper()) \
           and (kw.lstrip("(").rstrip(")") not in keywords_abb_key_cand) \
           and (ord(kw[0]) >= ord('A') and ord(kw[0]) <= ord('Z')) \
           and (len(kw) > 1):
            keywords_abb_key_cand.append(kw.lstrip("(").rstrip(")").rstrip(":").rstrip(","))

# convert to lower cases
# keywords_raw = [kw.lower() for kw in keywords_raw_]

# duplicate for values, and convert to lower cases
keywords_value = deepcopy(keywords_raw)

# convert to lower cases
keywords_value = [kw.lower() for kw in keywords_value]

# remove staring characters
keywords_value = [kw.lstrip('-') for kw in keywords_value]
keywords_value = [kw.rstrip('-') for kw in keywords_value]
keywords_value = [kw.lstrip(':') for kw in keywords_value]
keywords_value = [kw.rstrip(':') for kw in keywords_value]

# remove brakets "()"
for kc in keywords_abb_key_cand:
    kc_ = kc.lower()
    for kw in keywords_value:
        if kc_ in kw.replace('(', "").replace(')', "").split():
            kc_value = re.sub(r'\([^)]*\)', "", kw).rstrip(' ').lstrip(' ')
            if (kc_ not in kc_value) and \
               (kc_value[-1] != ")") and \
               (len(kc_value.split()) > 1) and \
               ((kc not in list(keywords_abb.keys())) or (kc in list(keywords_abb.keys()) and len(kc_value) < len(keywords_abb[kc]) and len(kc_value.split()) > 1)):
                kc_value = kc_value.replace("  ", " ")
                kc_value = ' '.join(kc_value.split(' ')[:-1] + [Lem.lemmatize(kc_value.split(' ')[-1])])
                keywords_abb.update({kc:kc_value})

keywords_value = [re.sub(r'\([^)]*\)', "", kw).rstrip(' ').lstrip(' ').lstrip(')').lstrip('(') for kw in keywords_value]

# remove unicodes –
unicode_keys = list(unicode_dict.keys())
for ukey in unicode_keys:
        keywords_value = [kw.replace(ukey, unicode_dict[ukey]) for kw in keywords_value]

# remove mathematics
keywords_value = [kw.replace("\\infty", "infinity") for kw in keywords_value]
keywords_value = [kw.replace("\\mathrm{", "") for kw in keywords_value]
keywords_value = [kw.replace("{", "") for kw in keywords_value]
keywords_value = [kw.replace("}", "") for kw in keywords_value]

# remove '"'
keywords_value = [kw.replace('"', "") for kw in keywords_value]

# reduce needless blanks.
keywords_value = [re.sub("\s+", " ", kw) for kw in keywords_value]
keywords_value = [kw.replace(" -", "-") for kw in keywords_value]
keywords_value = [kw.replace("- ", "-") for kw in keywords_value]
keywords_value = [kw.replace(" //", "//") for kw in keywords_value]
keywords_value = [kw.replace("// ", "//") for kw in keywords_value]

# create single-word keyword
keywords_single = [k for k in list(keywords_abb.keys())]
for kws in keywords_value:
    if len(kws.split(" ")) == 1 and len(kw) <= 4:
        keywords_single.append(kws)
    for kw in kws:
        if len(kw.split("-")) == 1 and len(kw) <= 4 and len(kw) > 1:
            keywords_single.append(kw)

keywords_single = list(np.unique(sorted(keywords_single)))

keywords_single_ = deepcopy(keywords_single)       
for kw in keywords_single_:
    if kw+"s" in keywords_single and len(kw) >= 4:
        print(kw+"s")
        keywords_single.remove(kw+"s")
    if kw[-1]+"ies" in keywords_single and kw[-1] == "y" and len(kw >=4):
        print(kw+"s")
        keywords_single.remove(kw+"s")

# create abbreviation dictionary
abb_sorted = dict(sorted(keywords_abb.items()))
with open(keyword_abb_name, "w") as j:
    json.dump(abb_sorted, j, ensure_ascii=False, indent=2)
    
# create "single" keywords list
with open(keyword_single_name, "w", encoding="utf-8-sig") as f:
    for keyword in keywords_single:
        f.write(f"{keyword}\n")

# create "value"
keywords_dict = dict(zip(keywords_raw, keywords_value))
with open(keyword_dict_name, "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)        

aerosols
aggregators
agrivoltaics
algorithms
barriers
bifurcations
buildings
characteristics
choppers
clouds
consumers
controllers
converters
correlations
costs
delays
dielectrics
diodes
dislocations
dynamics
economics
eigenvalues
electrolyzers
emissions
ensembles
experiments
failures
faults
feed-in-tariffs
forecasts
greenhouses
harmonics
heterojunctions
households
hurricanes
ibscs
igbts
imports
inverters
irradiances
measurements
metaheuristics
micro-grids
microgrids
microinverters
microturbines
mini-grids
modes
models
modules
multi-junctions
nano-grids
nanofluids
nanowires
optoelectronics
performances
perovskites
photodetectors
photodiodes
photovoltaics
prosumers
relays
renewables
resonances
scenarios
semiconductors
sensors
simulations
simulators
supercapacitors
systems
techno-economics
technoeconomics
thermodynamics
thermophotovoltaics
transients
trends
voltages
waves
wavelets


### manual refine `keywords_single`
* plural words not to be kept

In [7]:
# retrieve keywords_value
with open("keywords_dict.json", "r") as j:
    keywords_dict = json.load(j)
keywords_value = list(keywords_dict.values())
keywords_key = list(keywords_dict.keys())

# retreive keywords_single
keywords_single = np.genfromtxt("keywords_single_refine.txt", dtype="str", delimiter="\n", encoding='utf-8-sig') #utf-8-sig for \ufeff removal
keywords_single = [k.lower() for k in keywords_single]

# Convert plural to singular
Lem = WordNetLemmatizer()
keywords_plural = {}
keywords_value_ = []
for kws in keywords_value:
    if (kws not in keywords_single) and (len(kws.split(" ")) > 1):
        kws_0 = kws.split(" ")
        
        for kws_1 in kws_0:
            kws_2 = kws_1.split("-")
            for kw in kws_2:
                if (kw not in keywords_single) and (kw != Lem.lemmatize(kw)):
                    kws = kws.replace(kw, Lem.lemmatize(kw))
                    keywords_plural.update({kw: Lem.lemmatize(kw)})
        keywords_value_.append(kws)
    else:
        keywords_value_.append(kws)

keywords_value = deepcopy(keywords_value_)
keywords_value = [kw.lstrip(' ').rstrip(' ') for kw in keywords_value]

    
# create plural-singular dictionary
plural_sorted = dict(sorted(keywords_plural.items()))
with open(keyword_plural_name, "w") as j:
    json.dump(plural_sorted, j, ensure_ascii=False, indent=2)
    
# create "value"
keywords_dict = dict(zip(keywords_key, keywords_value))
with open(keyword_dict_name, "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)        

In [8]:
# retrieve keywords_value
with open("keywords_dict.json", "r") as j:
    keywords_dict = json.load(j)
keywords_value = list(keywords_dict.values())

# create dash dictionary
keywords_dash = []
for kw in keywords_value:
    kw_ = kw.split(" ")
    for kw__ in kw_:
        if ('-' in kw__) or ('//' in kw__):
            keywords_dash.append(kw__)
    
keywords_dash = np.unique(keywords_dash)

# create "dash" keywords dictionary - manual refinement required
keywords_dash_dict = dict(zip(keywords_dash, keywords_dash))
with open(keyword_dash_name, "w") as j:
    json.dump(keywords_dash_dict, j, ensure_ascii=False, indent=2)



### manual keywords-abb refinement
* incorrect names, typo errors
* unifying same words

### manual keyworkds-dash refinement
* incorrect names, typo errors
* unifying same words (ex. photo-voltaic and photovoltaic)

In [9]:
with open("keywords_plural.json", "r") as j:
    keywords_plural = json.load(j)

with open("keywords_abb_refine.json", "r") as j:
    keywords_abb_refine = json.load(j)

with open("keywords_dash_refine.json", "r") as j:
    keywords_dash_refine = json.load(j)
    

In [10]:
abb_refine_keys = list(keywords_abb_refine.keys())
dash_refine_keys = list(keywords_dash_refine.keys())

In [11]:
keywords_dash_refine2 = {}

plural_keys = list(keywords_plural.keys())

for dkey in dash_refine_keys:
    dvalues = keywords_dash_refine[dkey].split("; ")
    
    dvalues_abb = []
    dvalues_rms = []
    for dvalue in dvalues:
        dvalue_rem = deepcopy(dvalue)
        for akey in abb_refine_keys:
            if dvalue == akey.lower():
                dvalues.remove(dvalue)
                dvalues.append(keywords_abb_refine[akey])
            else:
                [dvalues_abb.append(keywords_abb_refine[d.upper()]) for d in dvalue.split("-") if d == akey.lower()]
                [dvalues_abb.append(keywords_abb_refine[d.upper()]) for d in dvalue.split("/") if d == akey.lower()]

    keywords_dash_refine2.update({dkey:dvalues+dvalues_abb})

with open("keywords_dash_refine2.json", "w") as j:
    json.dump(keywords_dash_refine2, j, ensure_ascii=False, indent=2)
                
            

### manual keyworkds_dash2 refinement
* incorrect abbs treatement

### apply keywords-dash on keywords_value
* (1) replace keywords_dash_refine.keys() to keywords_dash_refine.values()
* (2) if keywords_dash_refine2.key() has more than 2 elements, add them.

In [12]:
with open("keywords_dash_refine2.json", "r") as j:
    keywords_dash_refine2 = json.load(j)
    

keywords_value_ = []
for kws in keywords_value:
    dash_flag = 0
    kws_ = []
    for dkey in dash_refine_keys:
        
        if dkey in kws:
            kws_ += [kws.replace(dkey, keywords_dash_refine[dkey])]
            dash_flag += 1
            
            if len(keywords_dash_refine2[dkey]) > 1:
                [kws_.append(v) for v in keywords_dash_refine2[dkey][1:]]
                
    if dash_flag == 0:
        kws_ = [kws]
    
    keywords_value_.append(list(np.unique(kws_)))

keywords_dict = dict(zip(list(keywords_dict.keys()), keywords_value_))
with open(keyword_dict_name, "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)      

### find and extract dash and abbriviations
* create `keywords_dict_add.json`

In [14]:
%%time

abb_refine_keys = [k.lower() for k in np.unique(list(keywords_abb_refine.keys()))]
abb_refine_values = [v.lower() for v in np.unique(list(keywords_abb_refine.values()))]

keywords_value_add = []

for kw in keywords_value:
    kw_0 = [kw]
    kw_1 = kw.split(" ")
    kw_2 = kw.split("-")
    kw_3 = kw.split("/")
    
    keywords_value_add_ = []
    
    # "dash"
    for dash_keys in dash_refine_keys:
        if dash_keys in kw_1:
            [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]
        
        dash_keys_ = dash_keys.replace("-", "")
        if dash_keys_ in kw_1:
            if not (dash_keys_ in abb_refine_keys): # original abbriviation should be kept
                [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]

        dash_keys_ = dash_keys.split("-")
        if not (dash_keys_ in abb_refine_keys): # original abbriviation should be kept
            try:
                tmp = [kw_1.index(dash_key_) for dash_key_ in dash_keys_]
                if tmp == sorted(tmp) and (len(np.unique(tmp)) == len(tmp)):
                    [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]
            except ValueError:
                try:
                    tmp = [kw_2.index(dash_key_) for dash_key_ in dash_keys_]
                    if tmp == sorted(tmp) and (len(np.unique(tmp)) == len(tmp)):
                        [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]
                except ValueError:
                    pass
    
    # "abb"
    for abb_vals in abb_refine_values:
        if abb_vals in kw and \
           (abb_vals.replace(" ", "-") not in keywords_value_add) and \
           (abb_vals.replace("-", " ") not in keywords_value_add):
            keywords_value_add_.append(abb_vals)
    
    for abb_keys in abb_refine_keys:
        if abb_keys in (kw_0 + kw_1 + kw_2 + kw_3):
            keywords_value_add_.append(keywords_abb_refine[abb_keys.upper()])
    
    keywords_value_add.append(list(np.unique(keywords_value_add_)))

# create "additional" keywords dictionary
keywords_dict_add_dashabb = dict(zip(keywords_raw, keywords_value_add))
with open("keywords_dict_add_dashabb.json", "w") as j:
    json.dump(keywords_dict_add_dashabb, j, ensure_ascii=False, indent=2)  

CPU times: user 4min 16s, sys: 160 ms, total: 4min 17s
Wall time: 4min 17s


### Update `keywords_manual` on `keywords_dict` and `keywords_dict_add_dashabb`

In [140]:
with open("keywords_manual.json", "r") as j:
    keywords_manual = json.load(j)
    
with open("keywords_dict_add_dashabb.json", "r") as j:
    keywords_dict_add_dashabb = json.load(j)
    
with open("keywords_dict.json", "r") as j:
    keywords_dict = json.load(j)    

In [141]:
manual_keys = list(keywords_manual.keys())
dict_values = list(keywords_dict.values())
add_dashabb_values = list(keywords_dict_add_dashabb.values())

In [142]:
dict_values_update = []

for values in dict_values:
    value_new = []
    for value in values:
        for mkey in manual_keys:
            mkey_ = mkey.lower()
            value = value.replace(mkey_, keywords_manual[mkey])
        value_new.append(value)
    dict_values_update.append(value_new)
    
keywords_dict = dict(zip(keywords_raw, dict_values_update))
with open("keywords_dict2.json", "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)        

In [143]:
add_dashabb_values_update = []

for values in add_dashabb_values:
    value_new = []
    for value in values:
        for mkey in manual_keys:
            mkey_ = mkey.lower()
            value = value.replace(mkey_, keywords_manual[mkey])
        value_new.append(value)
    add_dashabb_values_update.append(value_new)
    
keywords_dict_add_dashabb = dict(zip(keywords_raw, add_dashabb_values_update))
with open("keywords_dict_add_dashabb2.json", "w") as j:
    json.dump(keywords_dict_add_dashabb, j, ensure_ascii=False, indent=2)      

### remove almost same words

In [165]:
import itertools

merged = list(itertools.chain.from_iterable([dict_values_update, add_dashabb_values_update]))
merged = list(itertools.chain.from_iterable(merged))
merged_u = np.unique(merged, return_counts=True)

words_u = merged_u[0]
   

df_words = pd.DataFrame({"keywords": merged_u[0],
                         "counts": merged_u[1]
                        }).sort_values("counts", ascending=False)
df_words.head(10)

,keywords,counts
10237,photovoltaic,1828
5297,generation,257
9100,neural network,178
8131,micro-grid,151
10434,photovoltaic system,120
4232,energy storage,119
8162,microgrid,115
6645,infrared,115
11627,pv-system,112
4293,energy-storage,106


In [145]:
# count = 0
# for word_u in words_u:
#     if (word_u.replace("-", " ") in words_u) and ("-" in word_u):
#         count += 1
#         print(count, word_u)

In [173]:
keywords_dash_update = {}

count = 0
for word in words_u:
    word_wo_dash = word.replace("-", "")
    if (word_wo_dash in words_u) and ("-" in word) :
        count += 1
        
        det_0 = all([len(w) > 5 for w in word.split(" ") if "-" in w])
        det_1 = all([len(w) >= 2 for w in word.split("-")]) 
#         print(count, word, det_0, det_1)
        
        if det_0 and det_1:
            keywords_dash_update.update({word_wo_dash: word})

# print(keywords_dash_update)

In [174]:
dash_update_keys = list(keywords_dash_update.keys())
dict_values_update_ = deepcopy(dict_values_update)

count = 0
for i, d in enumerate(dict_values_update):
    for key in dash_update_keys:
        if d == key:
            count += 1
            dict_values_update_[i] = keywords_dash_update[key]
    

### Update keywords based on `frequency` (top 300)